In [ ]:
! pip install -q pyspark==3.1.2 spark-nlp
! pip install bnlp_toolkit
! pip install -U bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bangla-stemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install -q pyspark==3.1.2 spark-nlp

In [ ]:
from bnlp import NLTKTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords
from bangla_stemmer.stemmer import stemmer
from gensim.models import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import emoji
import re
import nltk
import bnlp
import heapq
import json
import pickle
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from bnlp.corpus import stopwords, punctuations, letters, digits
#bnlp.download('stopwords')
#bnlp.download('wordnet')
#bnlp.download('averaged_perceptron_tagger')
#bnlp.download('maxent_ne_chunker')
#bnlp.download('words')
#bnlp.download('punkt')

In [ ]:
spark = sparknlp.start()

In [ ]:
pwd

'/content'

In [ ]:
#importing Dataset
dataset = pd.read_csv(r'/content/sample_data/Misogyny Classification - Sheet1.csv')
dataset.head()

,Text,Misogynistic
0,আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তা...,1
1,মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই,1
2,নষ্টা মহিলা,1
3,মেয়ে মানুষ মানে পুরাই ফেইক,1
4,বাজে মেয়েদের সাথে এরকমই হওয়া উচিত,1


In [ ]:
for col in dataset.columns:
    print(col, dataset[col].isnull().sum())

Text 0
Misogynistic 0


In [ ]:
dataset.shape

(2090, 2)

In [ ]:
dataset.dropna()
dataset.head()

,Text,Misogynistic
0,আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তা...,1
1,মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই,1
2,নষ্টা মহিলা,1
3,মেয়ে মানুষ মানে পুরাই ফেইক,1
4,বাজে মেয়েদের সাথে এরকমই হওয়া উচিত,1


In [ ]:
dataset.to_csv(r'/content/sample_data/Misogyny Classification - Sheet1.csv', encoding='utf-8')

In [ ]:
dataset = pd.read_csv(r'/content/sample_data/Misogyny Classification - Sheet1.csv')
dataset.drop('Unnamed: 0', axis=1, inplace=True)
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]

dataset.head()

,Text,Misogynistic
0,আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তা...,1.0
1,মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই,1.0
2,নষ্টা মহিলা,1.0
3,মেয়ে মানুষ মানে পুরাই ফেইক,1.0
4,বাজে মেয়েদের সাথে এরকমই হওয়া উচিত,1.0


In [ ]:
text = dataset['Text'].tolist()
m_lvl =dataset['Misogynistic'].tolist()
#s_lvl = dataset['Non-Misogynistic'].tolist()
print(text)
print(m_lvl)
#print(s_lvl)

['আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না?', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই হওয়া উচিত', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার হয় না', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক ভাবে আর এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন হয়', 'তারা বের করে চলবেন আর আমরা দেখলেই ধর্ষন', 'যৌবন তোমার লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের কোনো ভুমিকা নাই', 'মেয়ে হয়ে গলা উচা করে কথা বলেন কেন', 'সন্ধ্যার পর মেয়েদের বাসার বাইরে যাওয়া মোটেই উচিত না', 'বড় পদে নারীদের থাকা ভয়ানক ব্যাপার', 'জিনিস দেখানোর জন্য এরা ওড়না ঠিক মত রাখেনা', 'সব ছেলের দোষ না, ওর মতো নষ্ট মেয়েদের না ছেরে উপায় থাকে না।', 'নারীর শরীরটা নরম বলে নারী দুর্বল', 'আমাকে তোমার শরীর টা একটু চাটতে দিবা খুব ইচ্ছা করছে', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং করে তারা একেকটা পতিতা', 'এরা হল বেস্সা কোন জান্নাত পাবেনা', 'হালারপুত পোশাকের জন্য তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী 

In [ ]:
print(len(text))

2091


In [ ]:
#creating the corpus
#corpus = []
#corpus2 = []
#corpus3 = []
#corpus4 = []
#sentences = []
#for i in range(0,len(X)):
#  review = re.sub(r'\W',' ',str(X[i]))
#  review = review.lower()
#  review = re.sub(r'\s+[a-z]\s+',' ',review)
#  review = re.sub(r'^[a-z]\s+',' ',review)
#  review = re.sub(r'\s+',' ',review)
#  review = re.sub(r'\d',' ',review)
#  review = re.sub(r'\s+[a-z]\s+',' ',review)
#  review = re.sub(r'\s+',' ',review)
#  review = re.sub(r'^\s','',review)
#  review = re.sub(r'\s$','',review)
#  corpus.append(review)
#corpus2 = corpus
#corpus3 = corpus
#corpus4 = corpus
#sentences = corpus4
#print(corpus)
#print(len(corpus))

In [ ]:
def remove_numbers(text):
    text = re.sub(r"\d","",text)

print(text)

['আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না?', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই হওয়া উচিত', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার হয় না', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক ভাবে আর এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন হয়', 'তারা বের করে চলবেন আর আমরা দেখলেই ধর্ষন', 'যৌবন তোমার লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের কোনো ভুমিকা নাই', 'মেয়ে হয়ে গলা উচা করে কথা বলেন কেন', 'সন্ধ্যার পর মেয়েদের বাসার বাইরে যাওয়া মোটেই উচিত না', 'বড় পদে নারীদের থাকা ভয়ানক ব্যাপার', 'জিনিস দেখানোর জন্য এরা ওড়না ঠিক মত রাখেনা', 'সব ছেলের দোষ না, ওর মতো নষ্ট মেয়েদের না ছেরে উপায় থাকে না।', 'নারীর শরীরটা নরম বলে নারী দুর্বল', 'আমাকে তোমার শরীর টা একটু চাটতে দিবা খুব ইচ্ছা করছে', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং করে তারা একেকটা পতিতা', 'এরা হল বেস্সা কোন জান্নাত পাবেনা', 'হালারপুত পোশাকের জন্য তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী 

In [ ]:
# remove numbers
def remove_numbers(text):
    text = "".join(i for i in text if i in ["।"]
            or 2432 <= ord(i) <= 2531 or ord(i) == 32)
    text = " ".join(text.split())
    return text

print(text)

['আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না?', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই হওয়া উচিত', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার হয় না', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক ভাবে আর এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন হয়', 'তারা বের করে চলবেন আর আমরা দেখলেই ধর্ষন', 'যৌবন তোমার লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের কোনো ভুমিকা নাই', 'মেয়ে হয়ে গলা উচা করে কথা বলেন কেন', 'সন্ধ্যার পর মেয়েদের বাসার বাইরে যাওয়া মোটেই উচিত না', 'বড় পদে নারীদের থাকা ভয়ানক ব্যাপার', 'জিনিস দেখানোর জন্য এরা ওড়না ঠিক মত রাখেনা', 'সব ছেলের দোষ না, ওর মতো নষ্ট মেয়েদের না ছেরে উপায় থাকে না।', 'নারীর শরীরটা নরম বলে নারী দুর্বল', 'আমাকে তোমার শরীর টা একটু চাটতে দিবা খুব ইচ্ছা করছে', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং করে তারা একেকটা পতিতা', 'এরা হল বেস্সা কোন জান্নাত পাবেনা', 'হালারপুত পোশাকের জন্য তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী 

In [ ]:
import string
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords
from bnlp.corpus import stopwords, punctuations, letters, digits

print(stopwords)
print(punctuations)
print(letters)
print(digits)

raw_text = 'আমি ভাত খাই। ১২ ১৪ করিস না মাগি ১ +.... / ? .....'
new_text = raw_text.translate(str.maketrans('', '', string.punctuation))
result = remove_stopwords(raw_text, stopwords)

remove_digits = str.maketrans('', '', digits)
res = raw_text.translate(remove_digits)

print(res)

#def remove_punctuations(raw_text):
#  raw_text_nopunc = "".join([c for c in raw_text if c not in punctuations])
#  return raw_text_nopunc


#dataset['clean_text'] = dataset['Text'].apply(lambda x: remove_punctuations(x))
#dataset.head()


print(result)
print(new_text)
# ['ভাত', 'খাই', '।']

['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও'

In [ ]:
dataset.head()

,Text,Misogynistic
0,আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তা...,1.0
1,মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই,1.0
2,নষ্টা মহিলা,1.0
3,মেয়ে মানুষ মানে পুরাই ফেইক,1.0
4,বাজে মেয়েদের সাথে এরকমই হওয়া উচিত,1.0


In [ ]:
import string
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords
from bnlp.corpus import stopwords, punctuations, letters, digits

print(stopwords)
print(punctuations)
print(letters)
print(digits)

raw_text = 'আমি ভাত খাই। ১২ ১৪ করিস না মাগি ১ +.... / ? .....'
new_text = raw_text.translate(str.maketrans('', '', string.punctuation))
result = remove_stopwords(raw_text, stopwords)

remove_digits = str.maketrans('', '', digits)
res = raw_text.translate(remove_digits)

print(res)

#def remove_punctuations(raw_text):
#  raw_text_nopunc = "".join([c for c in raw_text if c not in punctuations])
#  return raw_text_nopunc


#dataset['clean_text'] = dataset['Text'].apply(lambda x: remove_punctuations(x))
#dataset.head()

def remove_digits(text):
  remove_digits = str.maketrans('','',digits)
  text = text.translate(remove_digits)
  return text




print(result)
print(new_text)
print(text)
# ['ভাত', 'খাই', '।']

['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও'

In [ ]:
# remove numbers
def remove_numbers(text):
    text = "".join(i for i in text if i in ["।"]
            or 2432 <= ord(i) <= 2531 or ord(i) == 32)
    text = " ".join(text.split())
    return text

In [ ]:
# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [ ]:
def bangla_stemmer(text): 
    temp =[]
    for x in text: 
        stmr = stemmer.BanglaStemmer()
        stm = stmr.stem(x)
        temp.append(stm)
    return temp

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained("lemma", "bn") \
    .setInputCols(["token"]) \
    .setOutputCol("lemma")


nlp_pipeline = Pipeline(stages=[document_assembler, tokenizer, lemmatizer])

light_pipeline = LightPipeline(nlp_pipeline.fit(
    spark.createDataFrame([[""]]).toDF("text")))

lemma download started this may take some time.
Approximate size to download 90.6 KB
[OK!]


In [ ]:
data_lst = []
def lemmatize_bangla(text):
  results = light_pipeline.fullAnnotate(text)
  for k, v in results[0].items():
    for item in range(len(v)):
      data_lst.append(v[item].result)
  return data_lst

In [ ]:
# lemmatize
# lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text

In [ ]:
print(text)

['আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না?', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই হওয়া উচিত', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার হয় না', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক ভাবে আর এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন হয়', 'তারা বের করে চলবেন আর আমরা দেখলেই ধর্ষন', 'যৌবন তোমার লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের কোনো ভুমিকা নাই', 'মেয়ে হয়ে গলা উচা করে কথা বলেন কেন', 'সন্ধ্যার পর মেয়েদের বাসার বাইরে যাওয়া মোটেই উচিত না', 'বড় পদে নারীদের থাকা ভয়ানক ব্যাপার', 'জিনিস দেখানোর জন্য এরা ওড়না ঠিক মত রাখেনা', 'সব ছেলের দোষ না, ওর মতো নষ্ট মেয়েদের না ছেরে উপায় থাকে না।', 'নারীর শরীরটা নরম বলে নারী দুর্বল', 'আমাকে তোমার শরীর টা একটু চাটতে দিবা খুব ইচ্ছা করছে', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং করে তারা একেকটা পতিতা', 'এরা হল বেস্সা কোন জান্নাত পাবেনা', 'হালারপুত পোশাকের জন্য তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী 

In [ ]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [ ]:
def join_text(text_list):
    temp =  ' '.join(text_list);
    return temp

In [ ]:
#emoji remove
temp = []
#for x in text:  
#    temp.append(remove_emoji(x))

#text = temp.copy()
#temp.clear()

In [ ]:
#punctuation remove
#print(temp)
for x in text:
    temp.append(remove_stopwords(x, punctuations))

text = temp.copy()
temp.clear()

for x in text:
    temp.append(join_text(x))

text = temp.copy()
temp.clear()

In [ ]:
# remove numbers
for x in text:
    temp.append(remove_numbers(x))


text = temp.copy()
temp.clear()

In [ ]:
print(text)

['আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই হওয়া উচিত', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার হয় না', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক ভাবে আর এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন হয়', 'তারা বের করে চলবেন আর আমরা দেখলেই ধর্ষন', 'যৌবন তোমার লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের কোনো ভুমিকা নাই', 'মেয়ে হয়ে গলা উচা করে কথা বলেন কেন', 'সন্ধ্যার পর মেয়েদের বাসার বাইরে যাওয়া মোটেই উচিত না', 'বড় পদে নারীদের থাকা ভয়ানক ব্যাপার', 'জিনিস দেখানোর জন্য এরা ওড়না ঠিক মত রাখেনা', 'সব ছেলের দোষ না ওর মতো নষ্ট মেয়েদের না ছেরে উপায় থাকে না', 'নারীর শরীরটা নরম বলে নারী দুর্বল', 'আমাকে তোমার শরীর টা একটু চাটতে দিবা খুব ইচ্ছা করছে', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং করে তারা একেকটা পতিতা', 'এরা হল বেস্সা কোন জান্নাত পাবেনা', 'হালারপুত পোশাকের জন্য তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী নার

In [ ]:
#remove stopwords4
def tokenize(self, text):
  temp = []
  for x in text:
      temp.append(remove_stopwords(x, stopwords))

  text = temp.copy()
  temp.clear()

In [ ]:
#stemmer 
def bangla_stemmer(text):
  temp =[]
  for x in text:
    stmr = stemmer.BanglaStemmer()
    stm = stmr.stem(x)
  for x in text:
    temp.append(bangla_stemmer(x))

  text = temp.copy()
  temp.clear()

In [ ]:
# rejoin the words as sentance
def join_text(text_list):
  for x in text:
      temp.append(join_text(x))

  text = temp.copy()
  temp.clear()
#print(text)

In [ ]:
# remove stopwords
for x in text:
    temp.append(remove_stopwords(x,stopwords))


text = temp.copy()
temp.clear()

In [ ]:
print(text)

[['পুরুষকে', 'সুখে', 'রাখার', 'বারোবাতারি', 'দরকার'], ['মন', 'ডায়', 'চাইতাছে', 'ছেরি', 'ডারে', 'থাপ্পড়', 'দিই'], ['নষ্টা', 'মহিলা'], ['মেয়ে', 'মানুষ', 'মানে', 'পুরাই', 'ফেইক'], ['বাজে', 'মেয়েদের', 'সাথে', 'এরকমই'], ['চরিত্রহীনা', 'নারী'], ['বুদ্ধিমান', 'মেয়েদের', 'দারা', 'সংসার'], ['বুদ্ধিমান', 'মেয়েরা', 'নিজেকে', 'অতিচালাক', 'এজন্য', 'সংসার', 'ভাঙ্গে'], ['সুন্দরিরা', 'বুদ্ধিহীন'], ['বের', 'চলবেন', 'দেখলেই', 'ধর্ষন'], ['যৌবন', 'লাল', 'টমেটো'], ['মেয়ে', 'মানুষ', 'মেয়ে', 'মানুষের', 'মত', 'থাকো'], ['প্রশাসনে', 'নারীদের', 'ভুমিকা'], ['মেয়ে', 'হয়ে', 'গলা', 'উচা', 'কথা'], ['সন্ধ্যার', 'মেয়েদের', 'বাসার', 'বাইরে'], ['বড়', 'পদে', 'নারীদের', 'ভয়ানক', 'ব্যাপার'], ['জিনিস', 'দেখানোর', 'ওড়না', 'মত', 'রাখেনা'], ['ছেলের', 'দোষ', 'নষ্ট', 'মেয়েদের', 'ছেরে', 'উপায়'], ['নারীর', 'শরীরটা', 'নরম', 'নারী', 'দুর্বল'], ['শরীর', 'টা', 'একটু', 'চাটতে', 'দিবা', 'ইচ্ছা'], ['ফালতু', 'ছেরি'], ['নির্লজ্জ', 'বেহায়া'], ['নাইকা', 'রা', 'একেকটা', 'পণ্য'], ['যেসব', 'মেয়ে', 'মডেলিং', 'একেকটা', 'পতিতা'], ['বেস্সা', 'জান্

In [ ]:
text2 = ([' '.join(i) for i in text])
print(text2)

['পুরুষকে সুখে রাখার বারোবাতারি দরকার', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন', 'বের চলবেন দেখলেই ধর্ষন', 'যৌবন লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের ভুমিকা', 'মেয়ে হয়ে গলা উচা কথা', 'সন্ধ্যার মেয়েদের বাসার বাইরে', 'বড় পদে নারীদের ভয়ানক ব্যাপার', 'জিনিস দেখানোর ওড়না মত রাখেনা', 'ছেলের দোষ নষ্ট মেয়েদের ছেরে উপায়', 'নারীর শরীরটা নরম নারী দুর্বল', 'শরীর টা একটু চাটতে দিবা ইচ্ছা', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং একেকটা পতিতা', 'বেস্সা জান্নাত পাবেনা', 'হালারপুত পোশাকের তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী নারী', 'ভাল মেয়ে নও সেক্স গার্ল', 'নারীদের শালীন কাপড় পরিধান শিক্ষানো ধর্ষণের মাত্রা কমে আসবে', 'ছেলেরা ফুটবল ম্যাচ মেয়েরা রান্নাঘরে পরচর্চা', 'নারী উপভোগ্য উপকরণ', 'নারীই মানুষের দুর্দশার', 'নারীদের ওপরে

In [ ]:
print(len(text2))

2091


In [ ]:
flat_list = []
for sublist in text:
    for item in sublist:
        flat_list.append(item)

print(flat_list)

['পুরুষকে', 'সুখে', 'রাখার', 'বারোবাতারি', 'দরকার', 'মন', 'ডায়', 'চাইতাছে', 'ছেরি', 'ডারে', 'থাপ্পড়', 'দিই', 'নষ্টা', 'মহিলা', 'মেয়ে', 'মানুষ', 'মানে', 'পুরাই', 'ফেইক', 'বাজে', 'মেয়েদের', 'সাথে', 'এরকমই', 'চরিত্রহীনা', 'নারী', 'বুদ্ধিমান', 'মেয়েদের', 'দারা', 'সংসার', 'বুদ্ধিমান', 'মেয়েরা', 'নিজেকে', 'অতিচালাক', 'এজন্য', 'সংসার', 'ভাঙ্গে', 'সুন্দরিরা', 'বুদ্ধিহীন', 'বের', 'চলবেন', 'দেখলেই', 'ধর্ষন', 'যৌবন', 'লাল', 'টমেটো', 'মেয়ে', 'মানুষ', 'মেয়ে', 'মানুষের', 'মত', 'থাকো', 'প্রশাসনে', 'নারীদের', 'ভুমিকা', 'মেয়ে', 'হয়ে', 'গলা', 'উচা', 'কথা', 'সন্ধ্যার', 'মেয়েদের', 'বাসার', 'বাইরে', 'বড়', 'পদে', 'নারীদের', 'ভয়ানক', 'ব্যাপার', 'জিনিস', 'দেখানোর', 'ওড়না', 'মত', 'রাখেনা', 'ছেলের', 'দোষ', 'নষ্ট', 'মেয়েদের', 'ছেরে', 'উপায়', 'নারীর', 'শরীরটা', 'নরম', 'নারী', 'দুর্বল', 'শরীর', 'টা', 'একটু', 'চাটতে', 'দিবা', 'ইচ্ছা', 'ফালতু', 'ছেরি', 'নির্লজ্জ', 'বেহায়া', 'নাইকা', 'রা', 'একেকটা', 'পণ্য', 'যেসব', 'মেয়ে', 'মডেলিং', 'একেকটা', 'পতিতা', 'বেস্সা', 'জান্নাত', 'পাবেনা', 'হালারপুত', 'পোশাকের', 'তর', 'মা'

In [ ]:
#creating histogram
from bnlp import NLTKTokenizer
bnltk = NLTKTokenizer()
word2count={}
senctence2count={}
for data in text2:
  words = bnltk.word_tokenize(data)
  for word in words:
    if word not in word2count.keys():
      word2count[word]=1
    else:
      word2count[word]+=1

for data in text2:
  sentence = bnltk.sentence_tokenize(data)
  for word in sentence:
    if word not in senctence2count.keys():
      senctence2count[word]=1
    else:
      senctence2count[word]+=1

print(word2count)  
print(len(word2count))   

print(senctence2count)  
print(len(senctence2count)) 

{'পুরুষকে': 16, 'সুখে': 1, 'রাখার': 6, 'বারোবাতারি': 1, 'দরকার': 35, 'মন': 13, 'ডায়': 1, 'চাইতাছে': 1, 'ছেরি': 2, 'ডারে': 1, 'থাপ্পড়': 1, 'দিই': 1, 'নষ্টা': 3, 'মহিলা': 44, 'মেয়ে': 103, 'মানুষ': 41, 'মানে': 13, 'পুরাই': 2, 'ফেইক': 1, 'বাজে': 10, 'মেয়েদের': 19, 'সাথে': 81, 'এরকমই': 2, 'চরিত্রহীনা': 1, 'নারী': 226, 'বুদ্ধিমান': 5, 'মেয়েদের': 86, 'দারা': 1, 'সংসার': 8, 'মেয়েরা': 68, 'নিজেকে': 15, 'অতিচালাক': 1, 'এজন্য': 1, 'ভাঙ্গে': 1, 'সুন্দরিরা': 2, 'বুদ্ধিহীন': 2, 'বের': 29, 'চলবেন': 3, 'দেখলেই': 4, 'ধর্ষন': 5, 'যৌবন': 2, 'লাল': 2, 'টমেটো': 1, 'মানুষের': 19, 'মত': 21, 'থাকো': 3, 'প্রশাসনে': 1, 'নারীদের': 85, 'ভুমিকা': 5, 'হয়ে': 78, 'গলা': 1, 'উচা': 1, 'কথা': 77, 'সন্ধ্যার': 1, 'বাসার': 5, 'বাইরে': 10, 'বড়': 20, 'পদে': 1, 'ভয়ানক': 1, 'ব্যাপার': 9, 'জিনিস': 10, 'দেখানোর': 2, 'ওড়না': 3, 'রাখেনা': 1, 'ছেলের': 6, 'দোষ': 21, 'নষ্ট': 18, 'ছেরে': 1, 'উপায়': 3, 'নারীর': 96, 'শরীরটা': 1, 'নরম': 1, 'দুর্বল': 4, 'শরীর': 11, 'টা': 23, 'একটু': 34, 'চাটতে': 1, 'দিবা': 2, 'ইচ্ছা': 15, 'ফালতু': 6, 'নি

In [ ]:
#finding out most frequent words
freq_words = heapq.nlargest(350,word2count,key=word2count.get)
print(freq_words)
print(len(freq_words))

['নারী', 'পর্দা', 'মেয়ে', 'নারীর', 'পুরুষ', 'মেয়েদের', 'পোশাক', 'নারীদের', 'ভালো', 'সাথে', 'পুরুষের', 'একটা', 'হয়ে', 'কথা', 'মা', 'নারীরা', 'প্রতিবাদ', 'মেয়েরা', 'একজন', 'বিয়ে', 'মহিলা', 'মানুষ', 'হতো', 'আপু', 'দিয়ে', 'নারীকে', 'দরকার', 'এক', 'একটু', 'যৌন', 'অধিকার', 'যায়', 'মেয়েটা', 'মেয়ের', 'বের', 'হিজাব', 'স্বামীর', 'সমাজে', 'ছাড়া', 'বোরকা', 'থাকলে', 'পড়ে', 'সুন্দর', 'সমাজের', 'পুরুষদের', 'মেয়ে', 'সম্মান', 'তোমরা', 'খারাপ', 'সমস্যা', 'হাত', 'টা', 'লাগে', 'কেনো', 'দেশে', 'হয়েছে', 'বোন', 'মুসলিম', 'করুন', 'ছোট', 'সমাজ', 'পর্দার', 'মত', 'দোষ', 'চায়', 'ভাই', 'উলঙ্গ', 'এরকম', 'বড়', 'মেয়েদের', 'মানুষের', 'তোমাদের', 'সময়', 'এসব', 'হিসেবে', 'হওয়া', 'করো', 'আইন', 'বিরুদ্ধে', 'নষ্ট', 'দেয়', 'ইসলামের', 'ছেলেদের', 'পোশাকের', 'চাহিদা', 'আসলে', 'মাগি', 'মর্যাদা', 'স্ত্রী', 'আরো', 'আল্লাহ', 'কাপড়', 'উচিৎ', 'পুরুষকে', 'স্বামী', 'ইসলাম', 'পোষাক', 'চললে', 'ছিলো', 'আল্লাহর', 'বাবা', 'মেয়েদেরকে', 'নিজেকে', 'ইচ্ছা', 'এতো', 'নারীদেরকে', 'চেয়ে', 'বিষয়', 'সবাই', 'বাসে', 'মাঝে', 'দুধ', 'অবস্থা', 'সবচেয়ে',

In [ ]:
X = dataset['Text'].tolist();
Y = dataset['Misogynistic'].tolist();
print(X)
print(Y)

['আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না?', 'মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই', 'নষ্টা মহিলা', 'মেয়ে মানুষ মানে পুরাই ফেইক', 'বাজে মেয়েদের সাথে এরকমই হওয়া উচিত', 'চরিত্রহীনা নারী', 'বুদ্ধিমান মেয়েদের দারা সংসার হয় না', 'বুদ্ধিমান মেয়েরা নিজেকে অতিচালাক ভাবে আর এজন্য সংসার ভাঙ্গে', 'সুন্দরিরা বুদ্ধিহীন হয়', 'তারা বের করে চলবেন আর আমরা দেখলেই ধর্ষন', 'যৌবন তোমার লাল টমেটো', 'মেয়ে মানুষ মেয়ে মানুষের মত থাকো', 'প্রশাসনে নারীদের কোনো ভুমিকা নাই', 'মেয়ে হয়ে গলা উচা করে কথা বলেন কেন', 'সন্ধ্যার পর মেয়েদের বাসার বাইরে যাওয়া মোটেই উচিত না', 'বড় পদে নারীদের থাকা ভয়ানক ব্যাপার', 'জিনিস দেখানোর জন্য এরা ওড়না ঠিক মত রাখেনা', 'সব ছেলের দোষ না, ওর মতো নষ্ট মেয়েদের না ছেরে উপায় থাকে না।', 'নারীর শরীরটা নরম বলে নারী দুর্বল', 'আমাকে তোমার শরীর টা একটু চাটতে দিবা খুব ইচ্ছা করছে', 'ফালতু ছেরি', 'নির্লজ্জ বেহায়া', 'নাইকা রা একেকটা পণ্য', 'যেসব মেয়ে মডেলিং করে তারা একেকটা পতিতা', 'এরা হল বেস্সা কোন জান্নাত পাবেনা', 'হালারপুত পোশাকের জন্য তর মা দায়ী', 'দুনিয়ার উত্তম উপভোগ্য উপকরণ পুণ্যবতী 

In [ ]:
#storing X and Y as pickle files
with open('text2.pickle','wb') as f:
  pickle.dump(X,f)

with open('Y.pickle','wb') as f:
  pickle.dump(Y,f)

#unpickling the dataset
with open('text2.pickle','rb') as f:
  X=pickle.load(f)

with open('Y.pickle','rb') as f:
  Y=pickle.load(f)

In [ ]:
#building bag of words model
BOWM = []
for data in text2:
  vector = []
  for word in freq_words:
    if word in bnltk.word_tokenize(data):
      vector.append(1)
    else:
      vector.append(0)
    BOWM.append(vector)
BOWM = np.asarray(BOWM)
print(BOWM)
print(len(BOWM))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
731850


In [ ]:
#TF Matrix

tf_matrix={}
for word in freq_words:
  doc_tf = []
  for data in text2:
    frequency = 0
    for w in bnltk.word_tokenize(data):
      if w == word:
        frequency += 1
    if len(bnltk.word_tokenize(data))!= 0:
      tf_word = frequency/len(bnltk.word_tokenize(data))
      doc_tf.append(tf_word)
    tf_matrix[word] = doc_tf

#print(tf_matrix)
#print(len(tf_matrix))

In [ ]:
len(text2)

2091

In [ ]:
len(Y)

2091

In [ ]:
#IDF Matrix

word_idfs = {}
for word in freq_words:
  doc_count=0
  for data in text2:
    if word in bnltk.word_tokenize(data):
      doc_count+=1
  word_idfs[word] = np.log((len(text)/doc_count)+1)

print(word_idfs)
print(len(word_idfs))

{'নারী': 2.3896754558057616, 'পর্দা': 3.0224743005016603, 'মেয়ে': 3.077449131226647, 'নারীর': 3.135951777135943, 'পুরুষ': 3.265483459884557, 'মেয়েদের': 3.2313557649810396, 'পোশাক': 3.30091653520441, 'নারীদের': 3.253967144679471, 'ভালো': 3.3253126841372826, 'সাথে': 3.3761105472324204, 'পুরুষের': 3.30091653520441, 'একটা': 3.350365661250028, 'হয়ে': 3.3253126841372826, 'কথা': 3.416109121157838, 'মা': 3.3631493140246254, 'নারীরা': 3.4025853069370036, 'প্রতিবাদ': 3.5016226621332507, 'মেয়েরা': 3.457892725338701, 'একজন': 3.681910784797843, 'বিয়ে': 3.927615100187982, 'মহিলা': 3.882032291744464, 'মানুষ': 3.975467177984681, 'হতো': 4.025821505167094, 'আপু': 4.025821505167094, 'দিয়ে': 4.0789488767441275, 'নারীকে': 4.0789488767441275, 'দরকার': 4.106649497412479, 'এক': 4.194849558903529, 'একটু': 4.135166556742356, 'যৌন': 4.194849558903529, 'অধিকার': 4.226127114688782, 'যায়': 4.135166556742356, 'মেয়েটা': 4.135166556742356, 'মেয়ের': 4.194849558903529, 'বের': 4.399658733113383, 'হিজাব': 4.39965873311338

In [ ]:
#TF-IDF calculation

tfidf_matrix = []
for word in tf_matrix.keys():
  tfidf = []
  for value in tf_matrix[word]:
    score = value*word_idfs[word]
    tfidf.append(score)
  tfidf_matrix.append(tfidf)

TFM = np.asarray(tfidf_matrix)
TFM = np.transpose(TFM)
print(TFM)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
print(len(TFM))

2089


In [ ]:
print(X[0])
print(Y[0])
print(X[2089])
print(Y[1606])

আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তাই না?
1.0
জীবনের প্রতিটি মুহূর্ত রঙিন হোক
0.0


In [ ]:
dataset.shape

(2091, 2)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2091 entries, 0 to 2090
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Text          2091 non-null   object 
 1   Misogynistic  2089 non-null   float64
dtypes: float64(1), object(1)
memory usage: 32.8+ KB


In [ ]:
dataset.dropna()

,Text,Misogynistic
0,আর পুরুষকে সুখে রাখার জন্য বারোবাতারি দরকার তা...,1.0
1,মন ডায় চাইতাছে ছেরি ডারে থাপ্পড় দিই,1.0
2,নষ্টা মহিলা,1.0
3,মেয়ে মানুষ মানে পুরাই ফেইক,1.0
4,বাজে মেয়েদের সাথে এরকমই হওয়া উচিত,1.0
...,...,...
2086,পুরুষ হইয়া জন্মানোটা পাপ হইয়া গেলো নাকি?,0.0
2087,ভীষণ অন্যায় হয়েছে,0.0
2088,যেমন কর্ম তেমন ফল একদম ঠিক আছে,0.0
2089,জীবনের প্রতিটি মুহূর্ত রঙিন হোক,0.0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2091 entries, 0 to 2090
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Text          2091 non-null   object 
 1   Misogynistic  2089 non-null   float64
dtypes: float64(1), object(1)
memory usage: 32.8+ KB


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    TFM, Y, test_size=0.1, random_state=42)

ValueError: ignored

In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
print(X_test)

In [ ]:
print(y_test)

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
#LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression
classifier_lr = LogisticRegression(random_state=42).fit(X_train, y_train)
y_prediction_lr = classifier_lr.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_lr = accuracy_score(y_test, y_prediction_lr)

print(accuracy_lr)

In [ ]:
from sklearn.metrics import precision_score
precision_lr = precision_score(y_test, y_prediction_lr, average='macro')
print(precision_lr)

In [ ]:
from sklearn.metrics import recall_score
recall_lr = recall_score(y_test, y_prediction_lr, average='macro')
print(recall_lr)

In [ ]:
from sklearn.metrics import f1_score
f1_score_lr = f1_score(y_test, y_prediction_lr, average='macro')
f1_score_lr2 = (2*precision_lr*recall_lr)/(precision_lr+recall_lr)
print(f1_score_lr)
print(f1_score_lr2)

In [ ]:
#K-NEAREST NEIGHBORS (K-NN)

from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier(n_neighbors = 5,metric = 'minkowski',p=2)
classifier_knn.fit(X_train,y_train)
y_prediction_knn = classifier_knn.predict(X_test)

In [ ]:
#from sklearn.metrics import accuracy_score

accuracy_knn = accuracy_score(y_test, y_prediction_knn)

print(accuracy_knn)

In [ ]:
#from sklearn.metrics import precision_score
precision_knn = precision_score(y_test, y_prediction_knn, average='macro')
print(precision_knn)

In [ ]:
#from sklearn.metrics import recall_score
recall_knn = recall_score(y_test, y_prediction_knn, average='macro')
print(recall_knn)

In [ ]:
#from sklearn.metrics import f1_score
f1_score_knn = f1_score(y_test, y_prediction_knn, average='macro')
f1_score_knn2 = (2*precision_knn*recall_knn)/(precision_knn+recall_knn)
print(f1_score_knn)
print(f1_score_knn2)

In [ ]:
#SUPPORT VECTOR MACHINE(SVM)

from sklearn.svm import SVC
classifier_svm = SVC(kernel = 'linear', random_state =0)
classifier_svm.fit(X_train,y_train)
y_prediction_svm = classifier_svm.predict(X_test)

In [ ]:
#from sklearn.metrics import accuracy_score

accuracy_svm = accuracy_score(y_test, y_prediction_svm)

print(accuracy_svm)

In [ ]:
#from sklearn.metrics import precision_score
precision_svm = precision_score(y_test, y_prediction_svm, average='macro')
print(precision_svm)

In [ ]:
#from sklearn.metrics import recall_score
recall_svm = recall_score(y_test, y_prediction_svm, average='macro')
print(recall_svm)

In [ ]:
#from sklearn.metrics import f1_score
f1_score_svm = f1_score(y_test, y_prediction_svm, average='macro')
f1_score_svm2 = (2*precision_svm*recall_svm)/(precision_svm+recall_svm)
print(f1_score_svm)
print(f1_score_svm2)

In [ ]:
#NAIVE BAYES
from sklearn.naive_bayes import GaussianNB
classifier_NB= GaussianNB()
classifier_NB.fit(X_train,y_train)
y_prediction_NB = classifier_NB.predict(X_test)

In [ ]:
accuracy_NB = accuracy_score(y_test, y_prediction_NB)

print(accuracy_NB)

In [ ]:
#from sklearn.metrics import precision_score
precision_NB = precision_score(y_test, y_prediction_NB, average='macro')
print(precision_NB)

In [ ]:
#from sklearn.metrics import recall_score
recall_NB = recall_score(y_test, y_prediction_NB, average='macro')
print(recall_NB)

In [ ]:
#from sklearn.metrics import f1_score
f1_score_NB = f1_score(y_test, y_prediction_NB, average='macro')
f1_score_NB2 = (2*precision_NB*recall_NB)/(precision_NB+recall_NB)
print(f1_score_NB)
print(f1_score_NB2)

In [ ]:
#DECISION TREE CLASSIFICATION

from sklearn.tree import DecisionTreeClassifier
classifier_dt = DecisionTreeClassifier(criterion = 'entropy',random_state=0)
classifier_dt.fit(X_train,y_train)
y_prediction_dt = classifier_dt.predict(X_test)

In [ ]:
accuracy_dt = accuracy_score(y_test, y_prediction_dt)

print(accuracy_dt)

In [ ]:
#from sklearn.metrics import precision_score
precision_dt = precision_score(y_test, y_prediction_dt, average='macro')
print(precision_dt)

In [ ]:
#from sklearn.metrics import recall_score
recall_dt = recall_score(y_test, y_prediction_dt, average='macro')
print(recall_dt)

In [ ]:
#from sklearn.metrics import f1_score
f1_score_dt = f1_score(y_test, y_prediction_dt, average='macro')
f1_score_dt2 = (2*precision_dt*recall_dt)/(precision_dt+recall_dt)
print(f1_score_dt)
print(f1_score_dt2)

In [ ]:
#RANDOM FOREST CLASSIFICATION

from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators=10, criterion = 'entropy', random_state=0)
classifier_rf.fit(X_train,y_train)
y_prediction_rf = classifier_rf.predict(X_test)

In [ ]:
accuracy_rf = accuracy_score(y_test, y_prediction_rf)

print(accuracy_rf)

In [ ]:
#from sklearn.metrics import precision_score
precision_rf = precision_score(y_test, y_prediction_rf, average='macro')
print(precision_rf)

In [ ]:
#from sklearn.metrics import recall_score
recall_rf = recall_score(y_test, y_prediction_rf, average='macro')
print(recall_rf)

In [ ]:
#from sklearn.metrics import f1_score
f1_score_rf = f1_score(y_test, y_prediction_rf, average='macro')
f1_score_rf2 = (2*precision_rf*recall_rf)/(precision_rf+recall_rf)
print(f1_score_rf)
print(f1_score_rf2)